In [1]:
import os
import zipfile
import numpy as np
import pandas as pd
import matplotlib
import torch
import matplotlib.pyplot as plt
# from tqdm import tqdm_notebook
%matplotlib inline 
# from google.colab.patches import cv2_imshow
from IPython.display import HTML #imports to play videos
from base64 import b64encode 
import cv2 as cv
#from skimage.measure import compare_ssim
from torchvision.datasets import ImageFolder
import glob
import time
from PIL import Image
from facenet_pytorch import MTCNN, InceptionResnetV1, extract_face
from tqdm import tqdm
from efficientnet_pytorch import EfficientNet
import math
import pickle
from functools import partial
from collections import defaultdict

from PIL import Image
from glob import glob
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
import cv2
import skimage.measure
import albumentations as A
from tqdm.notebook import tqdm 
#from albumentations.pytorch import ToTensor 
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision.models.video import mc3_18, r2plus1d_18

from facenet_pytorch import MTCNN



import sys
import sklearn

import cv2
import pandas as pd
import numpy as np

import plotly.graph_objs as go
from plotly.offline import iplot
from matplotlib import pyplot as plt
from torchvision import models, transforms
from torch.utils.data import WeightedRandomSampler

import matplotlib.pyplot as plt

plt.rc('font', size=14)
plt.rc('axes', labelsize=14, titlesize=14)
plt.rc('legend', fontsize=14)
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)


device = 'cuda'

DATA_FOLDER = os.getcwd()

BASE_PATH = os.getcwd()

ORI_CROPS = r'D:\FF\crops\original_sequences'
MANIP_CROPS = r'D:\FF\crops\manipulated_sequences'
CROPS_FOLDER = r'D:\FF\crops'
IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 10

c:\Users\Sheryl\AppData\Local\Programs\Python\Python312\Lib\site-packages\albumentations\__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.19 (you have 1.4.17). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
from glob import glob
import json
def get_videos_from_folder(folder_path):
    # List all files in the specified folder
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    if 'metadata.json' in files:
        files.remove('metadata.json')
    full_paths = [os.path.join(folder_path, f) for f in files]
    return full_paths

def get_videos_basenames_from_folder(folder_path):
    # List all files in the specified folder
    files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
    if 'metadata.json' in files:
        files.remove('metadata.json')
    return files

def get_original_with_fakes(root_dir):
    pairs = []
    for json_path in glob(os.path.join(root_dir, "*/metadata.json")):
        with open(json_path, "r") as f:
            metadata = json.load(f)
        for k, v in metadata.items():
            original = v.get("original", None)
            if v["label"] == "FAKE":
                pairs.append((original[:-4], k[:-4] ))

    return pairs


#metadata['aaqaifqrwn.mp4']['label']

def sample_weights(metadata):
    video_fake_real_labels = metadata.iloc[0]
    class_counts = video_fake_real_labels.value_counts()
    class_weights = 1 / class_counts
    return [1/class_counts[i] for i in video_fake_real_labels] # for undersampling

In [3]:
frames_per_video = 32
mtcnn = MTCNN(margin=20, select_largest=False, factor=0.5, device=device, post_process=True) # post_process=False if want human readable image
img_size = 224 # 256
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
class ImageTransform:
    def __init__(self, size, mean, std):
        self.data_transform = transforms.Compose([
                transforms.Resize((size, size), interpolation=Image.BILINEAR),
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
            ])

    def __call__(self, img):
        return self.data_transform(img)
    
transformer = ImageTransform(img_size, mean, std)

c:\Users\Sheryl\AppData\Local\Programs\Python\Python312\Lib\site-packages\facenet_pytorch\models\mtcnn.py:34: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.

c:\Users\Sheryl\A

In [4]:
train_dataset = ImageFolder(root=CROPS_FOLDER, transform=transformer) 
train_loader = DataLoader(train_dataset, shuffle=False, batch_size=64)

In [5]:
print(train_dataset.class_to_idx)

{'manipulated_sequences': 0, 'original_sequences': 1}


In [11]:
#pretrained_weights_path = 'noisy-student-efficientnet-b4.pth' # noisy student
#model = EfficientNet.from_name('efficientnet-b4')
#model = EfficientNet.from_name('efficientnet-b1')
#model.load_state_dict(torch.load(pretrained_weights_path, map_location=torch.device(device)))
#model._fc = nn.Linear(in_features=model._fc.in_features, out_features=1)
# Load the pre-trained EfficientNetV2B0 model (using EfficientNet)
env2b0_model = models.efficientnet_b0(weights='DEFAULT')
# Remove the classification head (last layer)
env2b0_model.classifier = nn.Identity()  # or you can use model.classifier = None

# Freeze all layers in the pre-trained model
for param in env2b0_model.parameters():
    param.requires_grad = False

# Custom top model
class CustomModel(nn.Module):
    def __init__(self, pretrained_model):
        super(CustomModel, self).__init__()
        self.pretrained = pretrained_model
        self.global_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.dense1 = nn.Linear(1280, 512)  # Adjust for your pretrained model's output
        self.batch_norm = nn.BatchNorm1d(512)
        self.dropout = nn.Dropout(0.2)
        self.dense2 = nn.Linear(512, 1)  # Output layer

    def forward(self, x):
        x = self.pretrained(x)
        x = self.dense1(x)
        x = self.batch_norm(x)
        x = self.dropout(x)
        x = self.dense2(x)
        x = torch.sigmoid(x).squeeze()      # Apply sigmoid activation
        return x

# Create the final model
model = CustomModel(env2b0_model)

In [12]:
for name, param in model.named_parameters():
    print(f"{name}: requires_grad={param.requires_grad}")

pretrained.features.0.0.weight: requires_grad=False
pretrained.features.0.1.weight: requires_grad=False
pretrained.features.0.1.bias: requires_grad=False
pretrained.features.1.0.block.0.0.weight: requires_grad=False
pretrained.features.1.0.block.0.1.weight: requires_grad=False
pretrained.features.1.0.block.0.1.bias: requires_grad=False
pretrained.features.1.0.block.1.fc1.weight: requires_grad=False
pretrained.features.1.0.block.1.fc1.bias: requires_grad=False
pretrained.features.1.0.block.1.fc2.weight: requires_grad=False
pretrained.features.1.0.block.1.fc2.bias: requires_grad=False
pretrained.features.1.0.block.2.0.weight: requires_grad=False
pretrained.features.1.0.block.2.1.weight: requires_grad=False
pretrained.features.1.0.block.2.1.bias: requires_grad=False
pretrained.features.2.0.block.0.0.weight: requires_grad=False
pretrained.features.2.0.block.0.1.weight: requires_grad=False
pretrained.features.2.0.block.0.1.bias: requires_grad=False
pretrained.features.2.0.block.1.0.weight: 

In [13]:

# train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=48, pin_memory=True)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-6)
loss_fn = nn.BCELoss()


train_loss_history = []
train_accuracy_history = []
val_loss_history = []
val_accuracy_history = []


In [14]:
class EarlyStopper:
    def __init__(self, patience=5, verbose=True):
        self.patience = patience
        self.counter = 0
        self.best_score = float('inf')  # Initialize to positive infinity
        self.verbose = verbose

    def early_stop(self, val_loss, model):
        if val_loss < self.best_score:
            self.best_score = val_loss  # Update best validation loss
            self.counter = 0  # Reset patience counter
            torch.save(model.state_dict(), 'best_model.pth')  # Save the best model
        else:
            self.counter += 1  # Increment patience counter
           
        if self.counter >= self.patience:
            if self.verbose:
                print("Early stopping...")
            return True
        return False

In [ ]:
epochs = 100
model = model.to(device)
earlystopper = EarlyStopper()
# mtcnn unable to detect gan/diffusion faces
#data = data.to(device)  # Move data to GPU if available
# half precision
# A6000 at desktop
for epoch in range(epochs): 
    running_loss = 0
    correct = 0
    total = 0
    model.train()
    for data, labels in train_loader:
        data = data.squeeze(0)
        labels = labels.squeeze(0)
        labels = labels.to(device).float()
        optimizer.zero_grad()  # clear previous gradients

        data = data.to(device).float()
        outputs = model(data)
        losses = loss_fn(outputs, labels)
        running_loss += losses.item()  # accumulate loss
        predicted = (outputs >= 0.5).float() # calculate if label is 0 or 1
        losses.backward()
        correct += (predicted == labels).sum().item() 
        total += labels.size(0)

    average_train_loss = running_loss / total
    average_train_accuracy = correct / total
    train_loss_history.append(average_train_loss)
    train_accuracy_history.append(average_train_accuracy)
    print(f"Epoch [{epoch}/{epochs}], Train Loss: {average_train_loss:.5f}, Train Accuracy: {average_train_accuracy:.5f}")

    
torch.save(model.state_dict(), 'model_ff.pth')

Epoch [0/100], Train Loss: 0.01166, Train Accuracy: 0.50042
Epoch [1/100], Train Loss: 0.01166, Train Accuracy: 0.49811
Epoch [2/100], Train Loss: 0.01166, Train Accuracy: 0.49954
Epoch [3/100], Train Loss: 0.01164, Train Accuracy: 0.50268
Epoch [4/100], Train Loss: 0.01166, Train Accuracy: 0.50062
Epoch [5/100], Train Loss: 0.01165, Train Accuracy: 0.50115
Epoch [6/100], Train Loss: 0.01168, Train Accuracy: 0.49802
Epoch [7/100], Train Loss: 0.01166, Train Accuracy: 0.50208
Epoch [8/100], Train Loss: 0.01166, Train Accuracy: 0.49855
Epoch [9/100], Train Loss: 0.01165, Train Accuracy: 0.50129
Epoch [10/100], Train Loss: 0.01165, Train Accuracy: 0.49882
Epoch [11/100], Train Loss: 0.01165, Train Accuracy: 0.50093


In [11]:
model.eval()
for data, label in train_loader:
        label = label.to(device)
        outputs=[]
        for face in data:
            face = face.to(device)
            output = model(face)
            outputs.append(output)
        answer = torch.mean(torch.stack(outputs), dim=0)
        print(answer, label)

tensor(0.5360, device='cuda:0', grad_fn=<MeanBackward1>) tensor([1], device='cuda:0')
tensor(0.5374, device='cuda:0', grad_fn=<MeanBackward1>) tensor([1], device='cuda:0')
tensor(0.5356, device='cuda:0', grad_fn=<MeanBackward1>) tensor([0], device='cuda:0')
tensor(0.5421, device='cuda:0', grad_fn=<MeanBackward1>) tensor([0], device='cuda:0')
tensor(0.8123, device='cuda:0', grad_fn=<MeanBackward1>) tensor([0], device='cuda:0')
tensor(0.6681, device='cuda:0', grad_fn=<MeanBackward1>) tensor([1], device='cuda:0')


KeyboardInterrupt: 

In [9]:
#pretrained_weights_path = 'noisy-student-efficientnet-b4.pth' # noisy student
#model = EfficientNet.from_name('efficientnet-b4')
#model.load_state_dict(torch.load(pretrained_weights_path, map_location=torch.device(device)))